# Example to use the custom Container for mono reconstruction

In [1]:
import os

from lstchain.io.lstcontainers import DL1ParametersContainer
from utils.gammalearn import load_model, load_camera_parameters

from ctapipe.utils import get_dataset_path
from ctapipe.io import HDF5TableWriter, HDF5TableReader
from ctapipe.calib import CameraCalibrator
from ctapipe.io import event_source

from astropy import units

import torch

## Options

In [ ]:
infile = get_dataset_path('gamma_test_large.simtel.gz')   

dl1_parameters_filename = 'dl1.h5'

allowed_tels = {1} # select LST1 only
max_events = 300 # limit the number of events to analyse in files - None if no limit

cal = CameraCalibrator()

## Load the model data

In [ ]:
if not os.path.isfile('lst_bootcamp_gl_data.tar.gz'):
    !wget https://gitlab.lapp.in2p3.fr/GammaLearn/GammaLearn/raw/master/share/lst_bootcamp_gl_data.tar.gz
    !tar xvzf lst_bootcamp_gl_data.tar.gz

In [ ]:
exps_path = 'lst_bootcamp_gl_data/'
camera_parameters_path = exps_path + 'camera_parameters.h5'

### Regression model

In [ ]:
exp_name = 'regression'
regression_network = load_model(exps_path, exp_name, 100, camera_parameters_path)

### Classification model

In [ ]:
exp_name = 'classification'
classification_network = load_model(exps_path, exp_name, 60, camera_parameters_path)

## R0 to DL2

In [ ]:
dl1_container = DL1ParametersContainer() 

In [ ]:
with HDF5TableWriter(filename=dl1_parameters_filename, group_name='events', overwrite=True) as writer:

    source = event_source(infile)
    source.allowed_tels = allowed_tels
    source.max_events = max_events

    for i, event in enumerate(source):
        if i%100==0:
            print(i)
        cal(event)

        for telescope_id, dl1 in event.dl1.tel.items():
            tel = event.inst.subarray.tels[telescope_id]
            camera = tel.camera

            image = dl1.image
            peakpos = dl1.pulse_time

            data = torch.tensor([image, peakpos], dtype=torch.float).unsqueeze(0)
            prediction = regression_network(data).squeeze(0).detach().numpy()
            particle_prediction = classification_network(data)
            particle = torch.max(particle_prediction, 1)[1]

            ## Fill container ##
            dl1_container.fill_mc(event)
            dl1_container.fill_event_info(event)
            dl1_container.set_mc_core_distance(event, telescope_id)
            dl1_container.set_source_camera_position(event, telescope_id)
            
            event.dl2.energy['gl'].energy = 10**prediction[0] * units.TeV
            event.dl2.shower['gl'].core_x = prediction[1] * units.km
            event.dl2.shower['gl'].core_y = prediction[2] * units.km
            event.dl2.shower['gl'].alt = prediction[3] * units.rad
            event.dl2.shower['gl'].az = prediction[4] * units.rad
            event.dl2.classification['gl'].prediction = particle.item()

            ## Save parameters for later training ##
            writer.write(camera.cam_id, [dl1_container, event.dl2.energy['gl'], event.dl2.shower['gl'],
                                        event.dl2.classification['gl']])
                
                

In [ ]:
# The file has been created
!ls -lsh dl1.h5

## Loading hdf5 file is very easy with pandas

In [ ]:
import pandas as pd
df = pd.read_hdf(dl1_parameters_filename, key='events/LSTCam')

In [ ]:
df